In [1]:
import numpy as np
import pandas as pd

## Data

In [2]:
n, m, nS = 8, 5, 2 # number of products, parts, scenarios

In [3]:
products = ['product ' + str(i + 1) for i in range(n)]
parts = ['part ' + str(j + 1) for j in range(m)]
scenarios = ['scenario ' + str(k + 1) for k in range(nS)]

In [4]:
density = pd.DataFrame(
    {
        'scenarios' : scenarios,
        'density' : [.5] * nS
    }
).set_index('scenarios')
density

,density
scenarios,
scenario 1,0.5
scenario 2,0.5


In [5]:
np.random.seed(42)

demands = pd.DataFrame(
    {
        'products' : np.repeat(products, nS),
        'scenarios' : scenarios * n,
        'demand' : np.array([np.random.binomial(10, .5, n) for k in range(nS)]).reshape(-1)
    }
).set_index(['products', 'scenarios'])

l = pd.DataFrame(
    {
        'products' : products,
        'additional cost' : np.random.randint(100, 200, n)
    }
).set_index('products') # l_i := additional cost to satify a unit of demand for product i

q = pd.DataFrame(
    {
        'products' : products,
        'selling price' : np.random.randint(200, 300, n)
    }
).set_index('products') # q_i := unit selling price of product i

b = pd.DataFrame(
    {
        'parts' : parts,
        'cost per unit' : np.random.randint(2, 5, m)
    }
).set_index('parts') # b_j := cost per unit of part j

s = pd.DataFrame(
    {
        'parts' : parts,
        'salvage' : np.random.randint(1, 4, m)
    }
).set_index('parts') # s_j := salvage values

for j in range(m):
    if s.iloc[j, 0] >= b.iloc[j, 0]:
        s.iloc[j, 0] = b.iloc[j, 0] - 1

A = pd.DataFrame(
    {
        'products' : np.repeat(products, m),
        'parts' : parts * n,
        'requirements' :  np.random.randint(0, 6, size=(n, m)).reshape(-1)
    }
).set_index(['products', 'parts'])

In [6]:
demands

demand
products  scenarios         
product 1 scenario 1       4
          scenario 2       8
product 2 scenario 1       6
          scenario 2       5
product 3 scenario 1       3
          scenario 2       3
product 4 scenario 1       3
          scenario 2       7
product 5 scenario 1       5
          scenario 2       6
product 6 scenario 1       2
          scenario 2       8
product 7 scenario 1       7
          scenario 2       4
product 8 scenario 1       4
          scenario 2       4

In [7]:
l

,additional cost
products,
product 1,175
product 2,157
product 3,121
product 4,188
product 5,148
product 6,190
product 7,158
product 8,141


In [8]:
q

,selling price
products,
product 1,291
product 2,259
product 3,279
product 4,214
product 5,261
product 6,261
product 7,246
product 8,261


In [9]:
b

,cost per unit
parts,
part 1,4
part 2,4
part 3,2
part 4,4
part 5,2


In [10]:
s

,salvage
parts,
part 1,3
part 2,3
part 3,1
part 4,1
part 5,1


In [11]:
A

requirements
products  parts               
product 1 part 1             1
          part 2             3
          part 3             0
          part 4             3
          part 5             5
product 2 part 1             1
          part 2             1
          part 3             0
          part 4             1
          part 5             4
product 3 part 1             1
          part 2             3
          part 3             3
          part 4             3
          part 5             3
product 4 part 1             4
          part 2             2
          part 3             5
          part 4             0
          part 5             3
product 5 part 1             1
          part 2             3
          part 3             1
          part 4             5
          part 5             5
product 6 part 1             5
          part 2             1
          part 3             3
          part 4             5
          part 5             4
product 7 part 1             1
          part 2             1
          part 3             3
          part 4             1
          part 5             1
product 8 part 1             5
          part 2             3
          part 3             5
          part 4             5
          part 5             3

In [12]:
pd.pivot_table(A.reset_index(), columns=['parts'], index=['products'])

requirements                            
parts           part 1 part 2 part 3 part 4 part 5
products                                          
product 1            1      3      0      3      5
product 2            1      1      0      1      4
product 3            1      3      3      3      3
product 4            4      2      5      0      3
product 5            1      3      1      5      5
product 6            5      1      3      5      4
product 7            1      1      3      1      1
product 8            5      3      5      5      3

## Symbol declaration

In [13]:
from gamspy import Container, Set, Parameter, Variable, Equation, Model, Sum, Sense

### Container

In [14]:
model = Container()

### Sets

In [15]:
i = Set(container=model, name='i', description='products', records=products)
j = Set(container=model, name='j', description='parts', records=parts)
k = Set(container=model, name='k', description='scenarios', records=scenarios)

### Parameters

In [16]:
A = Parameter(container=model, name='A', domain=[i, j], description='Requirements', records=A.reset_index())
A.records

,products,parts,value
0,product 1,part 1,1.0
1,product 1,part 2,3.0
2,product 1,part 3,0.0
3,product 1,part 4,3.0
4,product 1,part 5,5.0
5,product 2,part 1,1.0
6,product 2,part 2,1.0
7,product 2,part 3,0.0
8,product 2,part 4,1.0
9,product 2,part 5,4.0


In [17]:
pd.pivot_table(A.records, columns=['parts'], index=['products'])

value                            
parts     part 1 part 2 part 3 part 4 part 5
products                                    
product 1    1.0    3.0    0.0    3.0    5.0
product 2    1.0    1.0    0.0    1.0    4.0
product 3    1.0    3.0    3.0    3.0    3.0
product 4    4.0    2.0    5.0    0.0    3.0
product 5    1.0    3.0    1.0    5.0    5.0
product 6    5.0    1.0    3.0    5.0    4.0
product 7    1.0    1.0    3.0    1.0    1.0
product 8    5.0    3.0    5.0    5.0    3.0

In [18]:
l = Parameter(container=model, name='l', domain=[i],
              description='l_i := additional cost to satify a unit of demand for product i', records=l.reset_index())
l.records

,products,value
0,product 1,175.0
1,product 2,157.0
2,product 3,121.0
3,product 4,188.0
4,product 5,148.0
5,product 6,190.0
6,product 7,158.0
7,product 8,141.0


In [19]:
q = Parameter(container=model, name='q', domain=[i],
              description='q_i := unit selling price of product i', records=q.reset_index())
q.records

,products,value
0,product 1,291.0
1,product 2,259.0
2,product 3,279.0
3,product 4,214.0
4,product 5,261.0
5,product 6,261.0
6,product 7,246.0
7,product 8,261.0


In [20]:
b = Parameter(container=model, name='b', domain=[j],
              description='b_j := cost per unit of part j', records=b.reset_index())
b.records

,parts,value
0,part 1,4.0
1,part 2,4.0
2,part 3,2.0
3,part 4,4.0
4,part 5,2.0


In [21]:
s = Parameter(container=model, name='s', domain=[j],
              description='s_j := salvage values', records=s.reset_index())
s.records

,parts,value
0,part 1,3.0
1,part 2,3.0
2,part 3,1.0
3,part 4,1.0
4,part 5,1.0


In [22]:
d = Parameter(container=model, name='d', domain=[i, k], description='Demands', records=demands.reset_index())
d.records

,products,scenarios,value
0,product 1,scenario 1,4.0
1,product 1,scenario 2,8.0
2,product 2,scenario 1,6.0
3,product 2,scenario 2,5.0
4,product 3,scenario 1,3.0
5,product 3,scenario 2,3.0
6,product 4,scenario 1,3.0
7,product 4,scenario 2,7.0
8,product 5,scenario 1,5.0
9,product 5,scenario 2,6.0


In [23]:
p = Parameter(container=model, name='p', domain=[k], description='Density', records=density.reset_index())
p.records

,scenarios,value
0,scenario 1,0.5
1,scenario 2,0.5


### Variables

In [24]:
x = Variable(container=model, name='x', type='INTEGER', domain=[j], description='x_j := parts to be ordered (1st stage decision)')

In [25]:
y = Variable(container=model, name='y', type='INTEGER', domain=[j, k], description='y_j := Parts left in inventory')

In [26]:
z = Variable(container=model, name='z', type='INTEGER', domain=[i, k], description='z_i := number unit of part j produced')

### Equation

In [27]:
eq1 = Equation(container=model, name='eq1', domain=[j, k])
eq1[j, k] = y[j, k] == x[j] - Sum(domain=[i], expression=A[i, j] * z[i, k])

In [28]:
eq2 = Equation(container=model, name='eq2', domain=[j])
eq2[j] = x[j] >= 0

In [29]:
eq3 = Equation(container=model, name='eq3', domain=[j, k])
eq3[j, k] = y[j, k] >= 0

In [30]:
eq4 = Equation(container=model, name='eq4', domain=[i, k])
eq4[i, k] = z[i, k] >= 0

In [31]:
eq5 = Equation(container=model, name='eq5', domain=[i, k])
eq5[i, k] = z[i, k] <= d[i, k]

In [32]:
print(type(i))

<class 'gamspy._symbols.set.Set'>


### Objective function

In [33]:
Z = Sum(domain=[i], expression=(l[i] - q[i]) * z[i, k]) - Sum(domain=[j], expression=s[j] * y[j, k])
obj = Sum(domain=[j], expression=b[j] * x[j]) + Sum(domain=[k], \
    expression=(Z * p[k]))

In [34]:
model.getEquations()

[<Eq Equation `eq1` (0x1a89d1a6d10)>,
 <Geq Equation `eq2` (0x1a89d1a6b90)>,
 <Geq Equation `eq3` (0x1a89d136080)>,
 <Geq Equation `eq4` (0x1a89d1361d0)>,
 <Leq Equation `eq5` (0x1a89d134eb0)>]

In [35]:
# obj = Sum(domain=[j], expression=b[j] * x[j]) + Sum(domain=[k], \
#     expression=(Sum(domain=[i], expression=(l[i] - q[i]) * z[i, k]) + \
#                 Sum(domain=[j], expression=s[j] * y[j, k])) * p[k])

### Model

In [36]:
problem1 = Model(container=model, name='Problem_1', problem='MIP', sense=Sense.MIN, equations=model.getEquations(), objective=obj)

### Solve

In [37]:
import sys
problem1.solve(output=sys.stdout)

--- Job _gams_py_gjo0.gms Start 11/30/23 20:00:39 45.2.0 e4d2ee31 WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\Users\mayvp\AppData\Local\Programs\Python\Python310\Lib\site-packages\gamspy_base\gmsprmNT.txt
--- GAMS Parameters defined
    LP CPLEX
    MIP CPLEX
    RMIP CPLEX
    NLP CONOPT
    MCP PATH
    MPEC NLPEC
    RMPEC CONVERT
    CNS CONOPT
    DNLP CONOPT
    RMINLP CONOPT
    MINLP SBB
    QCP CONOPT
    MIQCP SBB
    RMIQCP CONOPT
    EMP CONVERT
    Input C:\Users\mayvp\AppData\Local\Temp\tmpo1gzjwe0\_gams_py_gjo0.gms
    Output C:\Users\mayvp\AppData\Local\Temp\tmpo1gzjwe0\_gams_py_gjo0.lst
    Save C:\Users\mayvp\AppData\Local\Temp\tmpo1gzjwe0\_gams_py_gcp0.g00
    ScrDir C:\Users\mayvp\AppData\Local\Temp\tmpo1gzjwe0\225a\
    SysDir C:\Users\mayvp\AppData\Local\Programs\Python\Python310\Lib\site-packages\gamspy_base\
    CurDir C:\Users\mayvp\AppData\Local\Temp\tmpo1gzjwe0\
    LogOption 3
    LogFile C:\Users\mayvp\AppData\Local\Temp\tmpo1gzjwe0\_gams_py_gjo0.log


In [38]:
x.records.set_index('j')

,level,marginal,lower,upper,scale
j,,,,,
part 1,55.0,4.0,0.0,inf,1.0
part 2,73.0,4.0,0.0,inf,1.0
part 3,61.0,2.0,0.0,inf,1.0
part 4,97.0,4.0,0.0,inf,1.0
part 5,119.0,2.0,0.0,inf,1.0


In [39]:
y.records.set_index(['j', 'k'])

level  marginal  lower  upper  scale
j      k                                               
part 1 scenario 1    0.0      -1.5    0.0    inf    1.0
       scenario 2    4.0      -1.5    0.0    inf    1.0
part 2 scenario 1   10.0      -1.5    0.0    inf    1.0
       scenario 2    0.0      -1.5    0.0    inf    1.0
part 3 scenario 1    0.0      -0.5    0.0    inf    1.0
       scenario 2   11.0      -0.5    0.0    inf    1.0
part 4 scenario 1    8.0      -0.5    0.0    inf    1.0
       scenario 2    0.0      -0.5    0.0    inf    1.0
part 5 scenario 1   14.0      -0.5    0.0    inf    1.0
       scenario 2    0.0      -0.5    0.0    inf    1.0

In [40]:
z.records.set_index(['i', 'k'])

level  marginal  lower  upper  scale
i         k                                               
product 1 scenario 1    4.0     -58.0    0.0    inf    1.0
          scenario 2    8.0     -58.0    0.0    inf    1.0
product 2 scenario 1    6.0     -51.0    0.0    inf    1.0
          scenario 2    5.0     -51.0    0.0    inf    1.0
product 3 scenario 1    3.0     -79.0    0.0    inf    1.0
          scenario 2    3.0     -79.0    0.0    inf    1.0
product 4 scenario 1    0.0     -13.0    0.0    inf    1.0
          scenario 2    0.0     -13.0    0.0    inf    1.0
product 5 scenario 1    5.0     -56.5    0.0    inf    1.0
          scenario 2    6.0     -56.5    0.0    inf    1.0
product 6 scenario 1    2.0     -35.5    0.0    inf    1.0
          scenario 2    1.0     -35.5    0.0    inf    1.0
product 7 scenario 1    7.0     -44.0    0.0    inf    1.0
          scenario 2    4.0     -44.0    0.0    inf    1.0
product 8 scenario 1    4.0     -60.0    0.0    inf    1.0
          scenario 2    4.0     -60.0    0.0    inf    1.0

In [41]:
problem1.objective_value

-2200.5